In [1]:
# استيراد المكتبات الضرورية
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# ----------------------------------------------
#  1. استيراد المكتبات
# ----------------------------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display  # لعرض الجداول في Jupyter

In [5]:
# ----------------------------------------------
#  2. قراءة البيانات من ملف CSV
# ----------------------------------------------
file_path = r'C:\Users\msmas\Desktop\دوري الذكاء الأصطناعي\DataSet\تصنيف اللاعبين.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig', sep=';')

In [7]:
df.head()

,اسم اللاعب,العمر,المركز,عدد التمريرات,دقة التمريرات (%),عدد التسديدات,عدد الأهداف,نسبة النجاح في المواجهات (%),السرعة القصوى (كم/س),المسافة المقطوعة (كم),معدل نبض القلب,نسبة التعب في نهاية الشوط,التقييم العام,تصنيف اللاعب
0,سعود القحطاني,23,وسط,91,67.96,11,4,47.24,31.31,9.19,125,48.07,48.31,موهوب
1,راكان المالكي,22,دفاع,46,79.73,3,3,65.58,26.62,6.56,153,23.19,45.19,موهوب
2,ناصر الشمري,19,دفاع,31,70.01,7,1,71.51,24.76,8.40,119,49.40,42.45,غير موهوب
3,خالد الغامدي,21,دفاع,30,86.79,2,3,81.95,32.90,7.12,174,34.03,50.20,موهوب
4,أحمد البلوي,23,دفاع,72,68.67,9,2,48.23,28.78,11.04,157,36.68,48.16,موهوب


In [9]:
# ----------------------------------------------
# 3. إعداد تصنيف اللاعب (موهوب أو غير موهوب)
# بناءً على التقييم العام (≥ 45 = موهوب)
# ----------------------------------------------
df['تصنيف اللاعب'] = df['التقييم العام'].apply(lambda x: 'موهوب' if x >= 45 else 'غير موهوب')

In [11]:
# ----------------------------------------------
#  4. تحديد المعايير المستخدمة في النموذج
# ----------------------------------------------
features = [
    'عدد التمريرات',
    'دقة التمريرات (%)',
    'عدد التسديدات',
    'عدد الأهداف',
    'نسبة النجاح في المواجهات (%)',
    'السرعة القصوى (كم/س)',
    'المسافة المقطوعة (كم)',
    'معدل نبض القلب',
    'نسبة التعب في نهاية الشوط'
]

X = df[features]
y = df['تصنيف اللاعب'].map({'موهوب': 1, 'غير موهوب': 0})
names = df['اسم اللاعب']
evaluation = df['التقييم العام']

In [13]:
# --------------------------------------------------------
# ✅ الجزء الأول: نموذج مستقطب المواهب 
# --------------------------------------------------------

In [15]:
# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
names_test = df.loc[y_test.index, 'اسم اللاعب']
evaluation_test = df.loc[y_test.index, 'التقييم العام']

In [17]:
# تدريب النموذج
model_recruiter = RandomForestClassifier(n_estimators=100, random_state=42)
model_recruiter.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
# التنبؤ والتقييم
y_pred = model_recruiter.predict(X_test)
y_proba = model_recruiter.predict_proba(X_test)[:, 1]

In [21]:
# إعداد جدول النتائج للمستقطب
results_recruiter = pd.DataFrame({
    'اسم اللاعب': names_test.values,
    'تصنيف اللاعب': np.where(y_pred == 1, 'موهوب', 'غير موهوب'),
    'درجة التقييم العام': evaluation_test.values.round(2),
    'نسبة الثقة بالتصنيف': (y_proba * 100).round(2),
    'توصية الاستقطاب': np.where(y_pred == 1, '✔️ نوصي باستقطاب اللاعب', '❌ لا يوصى بالاستقطاب')
}).reset_index(drop=True)

In [25]:
# عرض النتائج
print("🎯 نتائج نموذج المستقطب:")
display(results_recruiter.head(10))  # عرض أول 10 لاعبين

🎯 نتائج نموذج المستقطب:


,اسم اللاعب,تصنيف اللاعب,درجة التقييم العام,نسبة الثقة بالتصنيف,توصية الاستقطاب
0,فهد الرويلي,موهوب,50.23,98.0,✔️ نوصي باستقطاب اللاعب
1,فهد العنزي,موهوب,50.78,99.0,✔️ نوصي باستقطاب اللاعب
2,سعود الرويلي,غير موهوب,40.82,2.0,❌ لا يوصى بالاستقطاب
3,أحمد البلوي,موهوب,52.81,100.0,✔️ نوصي باستقطاب اللاعب
4,محمد العتيبي,غير موهوب,43.79,7.0,❌ لا يوصى بالاستقطاب
5,محمد القحطاني,موهوب,47.44,98.0,✔️ نوصي باستقطاب اللاعب
6,أحمد الزهراني,موهوب,49.35,100.0,✔️ نوصي باستقطاب اللاعب
7,راكان الرويلي,موهوب,46.82,72.0,✔️ نوصي باستقطاب اللاعب
8,محمد البلوي,موهوب,47.08,97.0,✔️ نوصي باستقطاب اللاعب
9,راكان العنزي,غير موهوب,44.00,4.0,❌ لا يوصى بالاستقطاب


In [29]:
# --------------------------------------------------------
# الجزء الثاني: (التحليل الفني مفصل)
# --------------------------------------------------------

In [31]:
# إعادة استخدام النموذج السابق
model_coach = model_recruiter 

In [33]:
# حساب المتوسطات لجميع اللاعبين
feature_means = X.mean()

In [35]:
# تحليل أداء كل لاعب
strengths = []
weaknesses = []

In [37]:
for _, row in X_test.iterrows():
    strong = []
    weak = []
    for col in features:
        if row[col] >= feature_means[col]:
            strong.append(col)
        else:
            weak.append(col)
    strengths.append("، ".join(strong))
    weaknesses.append("، ".join(weak))

In [39]:
# تجميع جدول تحليل المدرب
results_coach = pd.DataFrame({
    'اسم اللاعب': names_test.values,
    'تصنيف اللاعب': np.where(model_coach.predict(X_test) == 1, 'موهوب', 'غير موهوب'),
    'نقاط القوة': strengths,
    'نقاط تحتاج تطوير': weaknesses
}).reset_index(drop=True)

In [41]:
# عرض النتائج
print(" نتائج التحليل الفني:")
display(results_coach.head(10))  # عرض أول 10 لاعبين

 نتائج التحليل الفني:


,اسم اللاعب,تصنيف اللاعب,نقاط القوة,نقاط تحتاج تطوير
0,فهد الرويلي,موهوب,عدد التمريرات، عدد التسديدات، السرعة القصوى (ك...,دقة التمريرات (%)، عدد الأهداف، نسبة النجاح في...
1,فهد العنزي,موهوب,عدد التمريرات، دقة التمريرات (%)، عدد التسديدا...,نسبة النجاح في المواجهات (%)، نسبة التعب في نه...
2,سعود الرويلي,غير موهوب,دقة التمريرات (%)، عدد التسديدات، عدد الأهداف،...,عدد التمريرات، نسبة النجاح في المواجهات (%)، ا...
3,أحمد البلوي,موهوب,عدد التمريرات، دقة التمريرات (%)، عدد التسديدا...,عدد الأهداف، السرعة القصوى (كم/س)، المسافة الم...
4,محمد العتيبي,غير موهوب,عدد التمريرات، عدد التسديدات، عدد الأهداف، الس...,دقة التمريرات (%)، نسبة النجاح في المواجهات (%...
5,محمد القحطاني,موهوب,دقة التمريرات (%)، نسبة النجاح في المواجهات (%...,عدد التمريرات، عدد التسديدات، عدد الأهداف، معد...
6,أحمد الزهراني,موهوب,عدد التمريرات، نسبة النجاح في المواجهات (%)، ا...,دقة التمريرات (%)، عدد التسديدات، عدد الأهداف،...
7,راكان الرويلي,موهوب,عدد التسديدات، عدد الأهداف، نسبة النجاح في الم...,عدد التمريرات، دقة التمريرات (%)، المسافة المق...
8,محمد البلوي,موهوب,دقة التمريرات (%)، عدد التسديدات، عدد الأهداف،...,عدد التمريرات، نسبة النجاح في المواجهات (%)، ن...
9,راكان العنزي,غير موهوب,دقة التمريرات (%)، عدد الأهداف، السرعة القصوى ...,عدد التمريرات، عدد التسديدات، نسبة النجاح في ا...


In [43]:
# --------------------------------- نتائج دقة النموذج  -------------------------------- #  

In [45]:
from sklearn.metrics import accuracy_score, classification_report

In [49]:
# حساب دقة النموذج
accuracy = accuracy_score(y_test, y_pred)
classification_details = classification_report(y_test, y_pred, target_names=['غير موهوب', 'موهوب'])

In [51]:
# طباعة النتائج
accuracy_percentage = round(accuracy * 100, 2)
accuracy_percentage, classification_details

(96.35,
 '              precision    recall  f1-score   support\n\n   غير موهوب       0.96      0.95      0.95       787\n       موهوب       0.97      0.97      0.97      1213\n\n    accuracy                           0.96      2000\n   macro avg       0.96      0.96      0.96      2000\nweighted avg       0.96      0.96      0.96      2000\n')

In [53]:
# تحويل تقرير التصنيف النصي إلى جدول مرتب
from sklearn.metrics import classification_report
import pandas as pd
import io

In [55]:
# إعادة توليد تقرير التصنيف كنص
report_text = classification_report(y_test, y_pred, target_names=['غير موهوب', 'موهوب'])

In [57]:
# تحويل التقرير النصي إلى جدول باستخدام Pandas
report_df = pd.read_fwf(io.StringIO(report_text), index_col=0)

In [59]:
from IPython.display import display
display(report_df)

,precision,recall,f1-score,support
غير موهوب,0.96,0.95,0.95,787
موهوب,0.97,0.97,0.97,1213
accuracy,NaN,NaN,0.96,2000
macro avg,0.96,0.96,0.96,2000
weighted avg,0.96,0.96,0.96,2000


In [ ]:
# ---------------------------------   أختبار النموذج بمدخلات لاعب جديد   -------------------------------- #  

In [151]:
# ---------------------------------------------------
#  بيانات لاعب جديد (أدخل القيم يدويًا هنا)
# ---------------------------------------------------
new_player = pd.DataFrame([{
    'عدد التمريرات': 55,
    'دقة التمريرات (%)': 90,
    'عدد التسديدات': 4,
    'عدد الأهداف': 2,
    'نسبة النجاح في المواجهات (%)': 70,
    'السرعة القصوى (كم/س)': 30,
    'المسافة المقطوعة (كم)': 10,
    'معدل نبض القلب': 128,
    'نسبة التعب في نهاية الشوط': 35
}])


In [153]:
# ---------------------------------------------------
#  نموذج المستقطب
# ---------------------------------------------------
prediction_r = model_recruiter.predict(new_player)[0]
confidence_r = model_recruiter.predict_proba(new_player)[0][1] * 100

تصنيف_المستقطب = 'موهوب' if prediction_r == 1 else 'غير موهوب'
توصية_الاستقطاب = '✔️ نوصي باستقطاب اللاعب' if prediction_r == 1 else '❌ لا يوصى بالاستقطاب'

In [155]:
print(" [نموذج المستقطب]")
print("تصنيف اللاعب:", تصنيف_المستقطب)
print("نسبة الثقة:", round(confidence_r, 2), "%")
print("التوصية:", توصية_الاستقطاب)

 [نموذج المستقطب]
تصنيف اللاعب: موهوب
نسبة الثقة: 96.0 %
التوصية: ✔️ نوصي باستقطاب اللاعب


In [139]:
# --------------------------------------------
#  1. تجربة اللاعب على نموذج التصنيف
# --------------------------------------------
prediction = model_recruiter.predict(new_player)[0]
confidence = model_recruiter.predict_proba(new_player)[0][1] * 100


In [157]:
# ---------------------------------------------------
#  تحليل الأداء الفني 
# ---------------------------------------------------
# نستخدم نفس النموذج (model_recruiter) أو نسميه model_coach

feature_means = X.mean()  # حساب المتوسطات


In [159]:
# التنبؤ
prediction_c = model_recruiter.predict(new_player)[0]
تصنيف_المدرب = 'موهوب' if prediction_c == 1 else 'غير موهوب'

In [161]:
# تحليل الأداء الفني
نقاط_قوة = []
نقاط_تطوير = []

In [163]:
for col in features:
    if new_player.iloc[0][col] >= feature_means[col]:
        نقاط_قوة.append(col)
    else:
        نقاط_تطوير.append(col)

In [165]:
print("\n [نموذج المدرب]")
print("تصنيف اللاعب:", تصنيف_المدرب)
print(" نقاط القوة:", "، ".join(نقاط_قوة) if نقاط_قوة else "لا توجد")
print(" نقاط تحتاج تطوير:", "، ".join(نقاط_تطوير) if نقاط_تطوير else "لا توجد")


 [نموذج المدرب]
تصنيف اللاعب: موهوب
 نقاط القوة: عدد التمريرات، دقة التمريرات (%)، عدد الأهداف، نسبة النجاح في المواجهات (%)، السرعة القصوى (كم/س)، المسافة المقطوعة (كم)، نسبة التعب في نهاية الشوط
 نقاط تحتاج تطوير: عدد التسديدات، معدل نبض القلب


In [141]:
تصنيف_المستقطب = 'موهوب' if prediction == 1 else 'غير موهوب'
توصية_الاستقطاب = '✔️ نوصي باستقطاب اللاعب' if prediction == 1 else '❌ لا يوصى بالاستقطاب'

In [143]:
print(" [المستقطب]")
print("تصنيف اللاعب:", تصنيف_المستقطب)
print("نسبة الثقة:", round(confidence, 2), "%")
print("التوصية:", توصية_الاستقطاب)

🔍 [المستقطب]
تصنيف اللاعب: موهوب
نسبة الثقة: 96.0 %
التوصية: ✔️ نوصي باستقطاب اللاعب


In [145]:
# --------------------------------------------
# ✅ 2. تجربة اللاعب على نموذج المدرب
# --------------------------------------------

# نحسب المتوسطات من البيانات الأصلية
feature_means = X.mean()


In [147]:
# نحسب نقاط القوة والضعف
نقاط_قوة = []
نقاط_تطوير = []

for col in features:
    if new_player.iloc[0][col] >= feature_means[col]:
        نقاط_قوة.append(col)
    else:
        نقاط_تطوير.append(col)


In [149]:
# عرض التحليل الفني
print("\n [المدرب]")
print("نقاط القوة:", "، ".join(نقاط_قوة) if نقاط_قوة else "لا توجد")
print("نقاط تحتاج تطوير:", "، ".join(نقاط_تطوير) if نقاط_تطوير else "لا توجد")


 [المدرب]
نقاط القوة: عدد التمريرات، دقة التمريرات (%)، عدد الأهداف، نسبة النجاح في المواجهات (%)، السرعة القصوى (كم/س)، المسافة المقطوعة (كم)، نسبة التعب في نهاية الشوط
نقاط تحتاج تطوير: عدد التسديدات، معدل نبض القلب


In [167]:
import joblib
joblib.dump(model_recruiter, 'model_recruiter.pkl')

['model_recruiter.pkl']

In [91]:
joblib.dump(model, r'C:\Users\msmas\Documents\Player\player_model.pkl')

['C:\\Users\\msmas\\Documents\\Player\\player_model.pkl']

In [ ]:
#  نهاية الكود البرمجي 